<H1>Notebook for Deep Learning-based Binary Diagnosis of DR using the IDRID Disease Grading dataset</H1>

[![dataset](https://img.shields.io/badge/Open_in_Colab-orange)](https://githubtocolab.com/Sohambasu07/DR_2021/blob/master/notebooks/Diabetic_Retinopathy_detection_Binary_CNN.ipynb)
[![paper](https://img.shields.io/badge/Springer-Full_Paper-blue)](https://doi.org/10.1007/978-981-16-1543-6_16)
[![dataset](https://img.shields.io/badge/IDRID-Dataset-yellow)](https://ieee-dataport.org/open-access/indian-diabetic-retinopathy-image-dataset-idrid#files)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install tensorflow

#Imports

In [ ]:
import tensorflow as tf
import os
import numpy as np
import cv2 as cv
import random
from PIL import Image
import matplotlib.pyplot as plt
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, InputLayer, concatenate, Conv2D, MaxPooling2D, Flatten, Dense, UpSampling2D, Reshape, Dropout, Activation, Permute
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import plot_model as plot
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import Model
import pandas as pd
print(tf.__version__)

2.15.0


#Installing and logging into Wandb

In [ ]:
!pip install --upgrade wandb
!wandb login #Enter wandb login

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.1/267.1 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 10.9 MB/s eta 0:00:00
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
Aborted!


In [ ]:
!wandb login edbed8860733d3d377b3cc5ed23bdf45399b8c8c

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


#Paths

In [ ]:
drive_path = '/content/drive/MyDrive/DL_PROJECT_SEM6_DIABETIC_RETINOPATHY/'

trainds_path = '/IDRid/B. Disease Grading/2. Groundtruths/'
testds_path = '/IDRid/B. Disease Grading/2. Groundtruths/'
train_img_dir = '/IDRid/B. Disease Grading/1. Original Images/a. Training Set/'
test_img_dir = '/IDRid/B. Disease Grading/1. Original Images/b. Testing Set/'

#CSV to Pandas Dataframe

In [ ]:
#train_file = 'DR_Grading_Training Labels.csv'
train_file = 'a. IDRiD_Disease Grading_Training Labels.csv'
#test_file = 'DR_Grading_Testing Labels.csv'
test_file = 'b. IDRiD_Disease Grading_Testing Labels.csv'

traindf = pd.read_csv(drive_path + trainds_path + train_file, dtype=str)
traindf['Image name'] = traindf['Image name'].astype(str) + '.jpg'
testdf = pd.read_csv(drive_path + testds_path + test_file, dtype=str)
testdf['Image name'] = testdf['Image name'].astype(str) + '.jpg'
len(traindf)

413

In [ ]:
traindf.drop(traindf.filter(regex='Unnamed').columns, axis=1, inplace=True)

In [ ]:
traindf.tail(20)

,Image name,Retinopathy grade,Risk of macular edema
393,IDRiD_394.jpg,4,2
394,IDRiD_395.jpg,0,0
395,IDRiD_396.jpg,0,0
396,IDRiD_397.jpg,2,0
397,IDRiD_398.jpg,2,1
398,IDRiD_399.jpg,0,0
399,IDRiD_400.jpg,0,0
400,IDRiD_401.jpg,2,1
401,IDRiD_402.jpg,1,0
402,IDRiD_403.jpg,1,0


In [ ]:
testdf.tail(20)

,Image name,Retinopathy grade,Risk of macular edema
83,IDRiD_084.jpg,2,2
84,IDRiD_085.jpg,1,0
85,IDRiD_086.jpg,2,2
86,IDRiD_087.jpg,4,2
87,IDRiD_088.jpg,2,2
88,IDRiD_089.jpg,2,2
89,IDRiD_090.jpg,0,0
90,IDRiD_091.jpg,2,2
91,IDRiD_092.jpg,0,0
92,IDRiD_093.jpg,0,0


Converting the grade ino binary with the condition of 0,1 as 0 and 2,3,4 as 1

In [ ]:
traindf['Retinopathy grade'] = traindf['Retinopathy grade'].astype(int)  # Convert to integer type
testdf['Retinopathy grade'] = testdf['Retinopathy grade'].astype(int)

traindf['Retinopathy grade'] = traindf['Retinopathy grade'].apply(lambda x: 1 if x > 1 else 0)
testdf['Retinopathy grade'] = testdf['Retinopathy grade'].apply(lambda x: 1 if x > 1 else 0)


In [ ]:
testdf.tail(20)

,Image name,Retinopathy grade,Risk of macular edema
83,IDRiD_084.jpg,1,2
84,IDRiD_085.jpg,0,0
85,IDRiD_086.jpg,1,2
86,IDRiD_087.jpg,1,2
87,IDRiD_088.jpg,1,2
88,IDRiD_089.jpg,1,2
89,IDRiD_090.jpg,0,0
90,IDRiD_091.jpg,1,2
91,IDRiD_092.jpg,0,0
92,IDRiD_093.jpg,0,0


In [ ]:
traindf['Retinopathy grade'] = traindf['Retinopathy grade'].astype(str)  # Convert to integer type
testdf['Retinopathy grade'] = testdf['Retinopathy grade'].astype(str)

#Hyperparameters

In [ ]:
train_batch_size = 8
valid_batch_size = 8
test_batch_size = 8
img_size = 224

#Train ImageDataGenerator

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255.,
                                       horizontal_flip=True)#, validation_split = 0.20)

train_generator = train_datagen.flow_from_dataframe(
    dataframe = traindf,
    directory = drive_path + train_img_dir,#img_train_path, #drive_path + train_img_dir,
    x_col = 'Image name',
    y_col = 'Retinopathy grade',
    #subset = 'training',
    target_size = (img_size,img_size),
    class_mode = 'binary',
    batch_size = train_batch_size,
    shuffle = True
)



Found 356 validated image filenames belonging to 2 classes.


/usr/local/lib/python3.10/dist-packages/keras/src/preprocessing/image.py:1137: UserWarning: Found 57 invalid image filename(s) in x_col="Image name". These filename(s) will be ignored.
  warnings.warn(


#Test ImageDataGenerator

In [ ]:
test_datagen = ImageDataGenerator(rescale = 1./255.)# , validation_split = 0.20)

test_generator = test_datagen.flow_from_dataframe(
    dataframe = testdf,
    directory = drive_path + test_img_dir,#img_train_path, #drive_path + train_img_dir,
    x_col = 'Image name',
    y_col = 'Retinopathy grade',
    #subset = 'training',
    target_size = (img_size,img_size),
    class_mode = 'binary',
    batch_size = test_batch_size,
    shuffle = True
)

Found 103 validated image filenames belonging to 2 classes.


#Defining Neural Network Model

In [ ]:
def NNmodel():
  model = tf.keras.models.Sequential([
    #CONV1
    Conv2D(64, (3,3), activation='relu', padding ='same', input_shape=(img_size,img_size, 3)),
    Conv2D(64, (3,3), activation='relu', padding ='same'),
    MaxPooling2D(2,2),
    #Dropout(0.5),
    #CONV2
    Conv2D(128, (3,3), activation='relu', padding ='same'),
    Conv2D(128, (3,3), activation='relu', padding ='same'),
    MaxPooling2D(2,2),
    #Dropout(0.5),
    #CONV3
    Conv2D(256, (3,3), activation='relu', padding ='same'),
    Conv2D(256, (3,3), activation='relu', padding ='same'),
    Conv2D(256, (3,3), activation='relu', padding ='same'),
    Conv2D(256, (3,3), activation='relu', padding ='same'),
    Conv2D(256, (3,3), activation='relu', padding ='same'),
    MaxPooling2D(2,2),
    #Dropout(0.5),
    #CONV4
    Conv2D(512, (3,3), activation='relu', padding ='same'),
    Conv2D(512, (3,3), activation='relu', padding ='same'),
    Conv2D(512, (3,3), activation='relu', padding ='same'),
    Conv2D(512, (3,3), activation='relu', padding ='same'),
    Conv2D(512, (3,3), activation='relu', padding ='same'),
    MaxPooling2D(2,2),
    #Dropout(0.5),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1024, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
    ])
  model.summary()
  model.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.0001), metrics=['accuracy'])
  return model

In [ ]:
model = NNmodel()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 224, 224, 64)      1792      
                                                                 
 conv2d_1 (Conv2D)           (None, 224, 224, 64)      36928     
                                                                 
 max_pooling2d (MaxPooling2  (None, 112, 112, 64)      0         
 D)                                                              
                                                                 
 conv2d_2 (Conv2D)           (None, 112, 112, 128)     73856     
                                                                 
 conv2d_3 (Conv2D)           (None, 112, 112, 128)     147584    
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 56, 56, 128)       0         
 g2D)                                                   

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, concatenate, Flatten, Dense

def inception_module(x, filters):
    # 1x1 Convolution
    conv1x1 = Conv2D(filters=filters[0], kernel_size=(1, 1), padding='same', activation='relu')(x)

    # 3x3 Convolution
    conv3x3 = Conv2D(filters=filters[1], kernel_size=(3, 3), padding='same', activation='relu')(x)

    # 5x5 Convolution
    conv5x5 = Conv2D(filters=filters[2], kernel_size=(5, 5), padding='same', activation='relu')(x)

    # Max Pooling
    pool = MaxPooling2D(pool_size=(3, 3), strides=(1, 1), padding='same')(x)

    # Concatenate the outputs
    output = concatenate([conv1x1, conv3x3, conv5x5, pool], axis=-1)

    return output

def create_inception_model(input_shape, num_classes):
    inputs = Input(shape=input_shape)

    # Initial Convolution Layer
    x = Conv2D(filters=64, kernel_size=(7, 7), strides=(2, 2), padding='same', activation='relu')(inputs)
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same')(x)

    # First Inception Module
    x = inception_module(x, filters=[64, 128, 32])

    # Second Inception Module
    x = inception_module(x, filters=[128, 192, 96])

    # Third Inception Module
    x = inception_module(x, filters=[192, 208, 48])

    # Pooling Layer
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same')(x)

    # Flatten
    x = Flatten()(x)

    # Dense Layers
    x = Dense(512, activation='relu')(x)
    x = Dense(num_classes, activation='sigmoid')(x)

    model = tf.keras.Model(inputs=inputs, outputs=x)

    return model

# Define input shape and number of classes
input_shape = (224, 224, 3)  # Assuming 224x224 RGB images
num_classes = 1  # Adjust according to your classification task

# Create Inception model
inception_model = create_inception_model(input_shape, num_classes)

# Compile the model
inception_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Print model summary
inception_model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv2d_14 (Conv2D)          (None, 112, 112, 64)         9472      ['input_1[0][0]']             
                                                                                                  
 max_pooling2d_4 (MaxPoolin  (None, 56, 56, 64)           0         ['conv2d_14[0][0]']           
 g2D)                                                                                             
                                                                                                  
 conv2d_15 (Conv2D)          (None, 56, 56, 64)           4160      ['max_pooling2d_4[0][0]'] 

#Configuring Wandb project

In [ ]:
import wandb
wandb.init(project="diabetic_retinopathy_binary1", name='train-16cnn-idrid', sync_tensorboard=True)

KeyboardInterrupt: 

#Callback, Checkpoints and Early stopping

In [ ]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('val_accuracy')>=0.8500):
      print("\nReached 85.00% validation accuracy so cancelling training!")
      self.model.stop_training = True
callbacks1 = myCallback()

In [ ]:
checkpoint = ModelCheckpoint("/content/model2_best.hdf5", monitor='val_accuracy', verbose=1,
    save_best_only=True, mode='max', save_freq="epoch")

In [ ]:
earlystop = EarlyStopping(
    monitor='val_accuracy', min_delta=0, patience=5, verbose=1, mode='max',
    baseline=None, restore_best_weights=False
)

In [ ]:
callbacks = [
    tf.keras.callbacks.TensorBoard(log_dir=wandb.run.dir),
    checkpoint,
    callbacks1,
    earlystop,
    wandb.keras.WandbCallback()
]

#Training the model and saving metrics to history

In [ ]:
history = model.fit(train_generator,
                    validation_data = test_generator,
                    steps_per_epoch = train_generator.samples//train_batch_size,
                    epochs =  100,
                    validation_steps = test_generator.samples//valid_batch_size,
                    verbose = 1,
                    callbacks = [callbacks]
                    #callbacks = [callbacks1]
                    )

In [ ]:
history = inception_model.fit(train_generator,
                    validation_data = test_generator,
                    steps_per_epoch = train_generator.samples//train_batch_size,
                    epochs =  100,
                    validation_steps = test_generator.samples//valid_batch_size,
                    verbose = 1,
                    callbacks = [callbacks]
                    #callbacks = [callbacks1]
                    )

In [ ]:
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.applications import InceptionV3
# Load pre-trained InceptionV3 model
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(img_size, img_size, 3))

# Add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)

# Add a fully-connected layer
x = Dense(1024, activation='relu')(x)

# Add output layer
predictions = Dense(1, activation='sigmoid')(x)

# Compile the model
v3_model = tf.keras.Model(inputs=base_model.input, outputs=predictions)
v3_model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

# Define callbacks
checkpoint = ModelCheckpoint("/content/model_inception_best.hdf5", monitor='val_accuracy', verbose=1,
                             save_best_only=True, mode='max', save_freq="epoch")

earlystop = EarlyStopping(
    monitor='val_accuracy', min_delta=0, patience=5, verbose=1, mode='max',
    baseline=None, restore_best_weights=True
)

# Train the model
history = v3_model.fit(train_generator,
                    validation_data=test_generator,
                    steps_per_epoch=train_generator.samples // train_batch_size,
                    epochs=100,
                    validation_steps=test_generator.samples // valid_batch_size,
                    callbacks=[checkpoint, earlystop])

# Evaluate the model
test_loss, test_accuracy = v3_model.evaluate(test_generator)
print("Test Accuracy:", test_accuracy)

# Save the model
v3_model.save('/content/inception_model.h5')

#Saving the best weights

In [ ]:
v3_model.save(drive_path+'IDRID_v3_inceptionModel_iter0100.h5')

In [ ]:
# Make sure to import necessary libraries
import os
import random
from keras.preprocessing.image import load_img, img_to_array
from keras.models import Model
import numpy as np

# Concatenate the paths to get the full directory path
directory = drive_path + train_img_dir

# Get a list of all files in the directory
files = os.listdir(directory)

# Filter out only image files
image_files = [file for file in files if file.endswith(('jpg'))]

# Check if there are any image files in the directory
if image_files:
    # Select a random image file from the list
    rand_img_files = random.choice(image_files)
    # Get the full path of the randomly selected image
    rand_img_path = os.path.join(directory, rand_img_files)
    print("Randomly selected image:", rand_img_path)
else:
    print("No image files found in the directory:", directory)

# Load and preprocess the image
img = load_img(rand_img_path, target_size=(224, 224))  # Load image
x = img_to_array(img)  # Convert to numpy array
x = np.expand_dims(x, axis=0)  # Add batch dimension
x /= 255.0  # Normalize pixel values

# Define a new model that outputs activations of intermediate layers
layer_outputs = [layer.output for layer in v3_model.layers]  # Get activations of all layers
activation_model = Model(inputs=v3_model.input, outputs=layer_outputs)  # Create a new model

# Run the image through the network
try:
    successive_feature_maps = activation_model.predict(x)
    # Your visualization code here
except ValueError as e:
    print("ValueError during prediction:", e)


#Visualizing the model's inner layers

In [ ]:

# These are the names of the layers, so can have them as part of our plot
layer_names = [layer.name for layer in v3_model.layers[1:]]

# Now let's display our representations
for layer_name, feature_map in zip(layer_names, successive_feature_maps):
  if len(feature_map.shape) == 4:
    # Just do this for the conv / maxpool layers, not the fully-connected layers
    n_features = feature_map.shape[-1]  # number of features in feature map
    # The feature map has shape (1, size, size, n_features)
    size = feature_map.shape[1]
    # We will tile our images in this matrix
    display_grid = np.zeros((size, size * n_features))
    for i in range(n_features):
      # Postprocess the feature to make it visually palatable
      x = feature_map[0, :, :, i]
      x -= x.mean()
      x /= x.std()
      x *= 64
      x += 128
      x = np.clip(x, 0, 255).astype('uint8')
      # We'll tile each filter into this big horizontal grid
      display_grid[:, i * size : (i + 1) * size] = x
    # Display the grid
    scale = 20. / n_features
    plt.figure(figsize=(scale * n_features, scale))
    plt.title(layer_name)
    plt.grid(False)
    plt.imshow(display_grid, aspect='auto', cmap='viridis')

#Running inference on random images

In [ ]:
from google.colab import files
from keras.preprocessing import image

uploaded = files.upload()

for fn in uploaded.keys():

  # predicting images
  path = '/content/' + fn
  img = image.load_img(path, target_size=(224, 224))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)

  images = np.vstack([x])
  classes = v3_model.predict(images, batch_size=10)
  print(classes[0])
  if classes[0]>0.5:
    print(fn + " Presence of DR detected")
  else:
    print(fn + " No DR Detected")

#Evaluation on test set

In [ ]:
model_eval = load_model('/content/model2_best.hdf5')
hist_eval = model_eval.evaluate(
    test_generator,
    verbose = 1,
)
print(hist_eval)